In [7]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

print(os.listdir("/home/sam2220/Desktop/ML_projects/NY_taxi_fare_prediction/kaggle_challenge_dataset"))

['test.csv', '.~lock.sample_submission.csv#', 'sample_submission.csv', 'train.csv', 'GCP-Coupons-Instructions.rtf']


In [8]:
#read train and test dataset 
TRAIN_PATH = "/home/sam2220/Desktop/ML_projects/NY_taxi_fare_prediction/kaggle_challenge_dataset/train.csv"
TEST_PATH = "/home/sam2220/Desktop/ML_projects/NY_taxi_fare_prediction/kaggle_challenge_dataset/test.csv"
with open(TRAIN_PATH) as file:
    len_train = len(file.readlines())

print(len_train)

55423857


In [9]:
train_temp = pd.read_csv(TRAIN_PATH, nrows=5)
print("trainig dataset head:")
print(train_temp)
#test_temp = pd.read_csv(TEST_PATH, nrows=5)
#print("test dataset head:")
#print(test_temp)

trainig dataset head:
                             key  fare_amount          pickup_datetime  \
0    2009-06-15 17:26:21.0000001          4.5  2009-06-15 17:26:21 UTC   
1    2010-01-05 16:52:16.0000002         16.9  2010-01-05 16:52:16 UTC   
2   2011-08-18 00:35:00.00000049          5.7  2011-08-18 00:35:00 UTC   
3    2012-04-21 04:30:42.0000001          7.7  2012-04-21 04:30:42 UTC   
4  2010-03-09 07:51:00.000000135          5.3  2010-03-09 07:51:00 UTC   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.844311        40.721319         -73.841610         40.712278   
1        -74.016048        40.711303         -73.979268         40.782004   
2        -73.982738        40.761270         -73.991242         40.750562   
3        -73.987130        40.733143         -73.991567         40.758092   
4        -73.968095        40.768008         -73.956655         40.783762   

   passenger_count  
0                1  
1                1  
2      

In [14]:
#chunksize = 1000

train_types = {'fare_amount': 'float32',
              'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

test_types =  {'pickup_datetime': 'str', 
              'pickup_longitude': 'float32',
              'pickup_latitude': 'float32',
              'dropoff_longitude': 'float32',
              'dropoff_latitude': 'float32',
              'passenger_count': 'uint8'}

train_cols = list(train_types.keys())
test_cols = list(test_types.keys())

def clean_df(PATH, cols, datatypes, chunksize):
    if PATH == TEST_PATH:
        df_list = []
        for chunk in pd.read_csv(PATH, usecols = cols, dtype=datatypes, chunksize=chunksize):
            chunk['pickup_datetime'] = chunk['pickup_datetime'].str.slice(0, 16)
            chunk['pickup_datetime'] = pd.to_datetime(chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')

            df_list.append(chunk)
            #print(len(df_list))
        return pd.concat(df_list)
    
    elif(PATH == TRAIN_PATH):
        X_df_list = []
        y_df_list = []
        for chunk in pd.read_csv(PATH, usecols = cols, dtype=datatypes, chunksize=chunksize):            
            chunk['pickup_datetime'] = chunk['pickup_datetime'].str.slice(0, 16)
            chunk['pickup_datetime'] = pd.to_datetime(chunk['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
            
            y_df_list.append(chunk['fare_amount'])
            chunk.drop(labels = 'fare_amount', axis=1, inplace=True)
            X_df_list.append(chunk)
        
        X_df = pd.concat(X_df_list)
        y_df = pd.concat(y_df_list)
        return  X_df, y_df

#insample dataset
train_target, train_label = clean_df(TRAIN_PATH, train_cols, train_types, chunksize = 5000000)
n_train = len(train_label)
print(n_train)

#out of samples dataset
test_dataset = clean_df(TEST_PATH, test_cols, test_types, chunksize = 2000)
n_test = len(test_dataset)
print(n_test)

55423856
9914


In [ ]:
#separate label and features from training dataset
'''
features = test_cols
X = train_dataset.loc[:, features].values
y = train_dataset.iloc[:, 0].values
print(X.shape())
print(y.shape())
'''

In [ ]:
X = ['a', 'b', 'c']
